<a href="https://colab.research.google.com/github/yijiao886/learning.../blob/main/_downloads/070179efc13bd796c5dd4af7bf52d5b9/intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [16]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

就像做菜需要准备食材一样，训练AI也需要准备数据。
Dataset (食材库):
是什么？ Dataset 就像一个巨大的食材仓库。它储存了我们所有的原材料——成千上万张衣服图片 (样本)，以及每张图片对应的正确名称 (标签)。
在代码中: datasets.FashionMNIST(...) 就是PyTorch官方帮我们准备好的一个现成的、装满了衣服图片和标签的大仓库。train=True表示我们拿的是用于“学习”的训练食材库，train=False是用于“考试”的测试食材库。download=True 意味着如果你的厨房里还没有这个食材库，它会自动帮你从网上下载。

In [17]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


DataLoader (配菜员):
是什么？ DataLoader 就像一个高效的配菜员。我们的大厨（模型）一次吃不下整个食材库的6万张图片，而且如果一张一张地喂，效率太低。
干什么？ 配菜员的工作就是从食材库 (Dataset) 中，按照我们规定的份量（batch_size），一盘一盘地把菜配好。比如 batch_size = 64，就是告诉配菜员：“请每次给我拿64张图片和它们对应的64个标签，打包成一盘菜。”
有什么用？ 自动打包（批处理）、洗牌（shuffle=True，打乱顺序防止大厨养成坏习惯）、多线程备菜（num_workers），极大地提高了备菜效率。
for X, y in test_dataloader: (上菜):
这行代码就是在说：“配菜员，把下一盘菜端上来！”
X 就是那一盘64张图片的数据，它的形状是 [64, 1, 28, 28]，意思是 [64张图片, 每张是1个通道(黑白的), 高28像素, 宽28像素]。
y 就是那64个对应的正确标签。
小结： 这一步的核心就是把原始数据整理成模型可以**高效“食用”**的一批一批（batch）的格式。

In [18]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


现在食材准备好了，我们需要给“AI大厨”一本菜谱（NeuralNetwork 类），告诉它应该如何一步步地处理食材（图片）。
class NeuralNetwork(nn.Module): (菜谱封面):
是什么？ 我们正在定义一本名叫NeuralNetwork的菜谱。所有PyTorch的菜谱都必须是nn.Module的“徒弟”，继承它的“厨艺精髓”。
def __init__(self): (准备厨具):
是什么？ 这是菜谱的“厨具准备”章节。它列出了这道菜需要用到的所有厨具（网络层）。
self.flatten = nn.Flatten(): 准备一个叫flatten的**“擀面杖”**。它的作用是把输入的28x28像素的二维图片，“擀平”成一个784像素长的一维长条。
self.linear_relu_stack = nn.Sequential(...): 准备一个叫linear_relu_stack的**“多功能料理机”**。nn.Sequential就像一个流水线，它会按照顺序自动执行放进去的所有厨具。
nn.Linear(28*28, 512): 一个**“搅拌机”**（全连接层）。它把784个像素点的信息“搅拌混合”，输出512个新的特征。
nn.ReLU(): 一个**“过滤器”**（激活函数）。它把搅拌后的结果中所有负数都变成0，只保留正数信息。
...又一个搅拌机和过滤器...
nn.Linear(512, 10): 最后一个**“分拣机”**。它把512个特征最终“分拣”成10个输出值，每个值对应这件衣服是10个类别之一（T恤、裤子...）的可能性得分。
def forward(self, x): (烹饪步骤):
是什么？ 这是菜谱的“烹饪步骤”章节。它清晰地描述了处理一份食材x（一张图片）的先后顺序。
x = self.flatten(x): “第一步，用擀面杖把图片擀平。”
logits = self.linear_relu_stack(x): “第二步，把擀平的面条送入多功能料理机进行处理。”
return logits: “最后，端出料理机处理好的、包含10个评分的最终成品（logits）。”
.to(device) (选择灶台):
device 指的是计算设备。.to(device) 就是把我们的厨具（模型）和食材（数据）都搬到GPU这个“专业级电磁炉”上，让计算速度飞快。如果没GPU，就用CPU这个“普通煤气灶”。
小结： 这一步就是用代码定义神经网络的结构和数据流动的路径。


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

现在菜谱和食材都有了，我们要开始教“AI大厨”做菜了。这个过程就是训练。
loss_fn = nn.CrossEntropyLoss() (美食评论家 - 损失函数):
是什么？ 我们请来了一位非常严格的**“美食评论家”（损失函数）**。
干什么？ 每当大厨做出一道菜（给出预测），评论家就会对比大厨的成品（pred）和原始菜谱的要求（y，即正确标签），然后给出一个评分（loss）。
如果大厨做的和菜谱要求一模一样，评分就很低（比如0.01），说明差距小。
如果大厨把T恤看成了裤子，评分就会很高（比如2.3），说明差距大。
这个评分就是我们之前说的“损失（Loss）”。
optimizer = torch.optim.SGD(...) (烹饪导师 - 优化器):
是什么？ 这位是**“烹饪导师”（优化器）**。
干什么？ 他会看着美食评论家给出的高分（高Loss），然后根据这个差评，告诉大厨**应该如何调整自己的烹饪手法（模型的参数/权重）**才能做得更好。
model.parameters(): 导师要指导的对象，就是大厨的所有“烹饪手法”（模型的所有参数）。
lr=1e-3 (学习率): 这是导师的教学风格。lr是learning rate（学习率）的缩写。它决定了每次调整手法的幅度。1e-3就是0.001。
学习率太高，就像导师说：“你这盐放错了！下次把整罐盐都倒掉！”——调整过猛，容易矫枉过正。
学习率太低，就像导师说：“盐稍微多了那么一丁点儿，下次减少0.01克试试。”——调整太慢，猴年马月才能学会。

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

train 函数 (一轮学习):
model.train(): “大厨，进入学习模式！”
for batch, (X, y) in enumerate(dataloader): “配菜员，开始一盘一盘地上菜！”
pred = model(X): “大厨，根据你现在的手法，做一盘菜。”
loss = loss_fn(pred, y): “评论家，快来打分！”
loss.backward() (反思过程): 这是最神奇的一步，叫做反向传播。大厨（PyTorch的自动求导引擎）会根据评论家给出的差评（Loss），自动反思出到底是哪个厨具（层）、哪个手法（参数）出了问题，以及应该朝哪个方向调整。
optimizer.step() (调整手法): “导师，根据反思结果，动手调整一下大厨的手法！” 优化器会根据学习率，对模型的参数进行微调。
optimizer.zero_grad() (清空笔记): “好了，这次的错误已经纠正了，把之前的‘反思笔记’（梯度）清空，准备做下一盘菜。”

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

test 函数 (阶段性考试):
model.eval(): “大厨，进入考试模式！不许再学了。”
with torch.no_grad(): “考试期间，关闭‘反思’功能（不计算梯度），节省体力。”
循环遍历测试食材库，只进行预测和评分，但不进行backward()和step()。
最后计算出在整个“考场”上的平均分（Avg loss）和正确率（Accuracy）。

In [22]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300410  [   64/60000]
loss: 2.289523  [ 6464/60000]
loss: 2.272384  [12864/60000]
loss: 2.267132  [19264/60000]
loss: 2.254844  [25664/60000]
loss: 2.215510  [32064/60000]
loss: 2.227537  [38464/60000]
loss: 2.180635  [44864/60000]
loss: 2.180346  [51264/60000]
loss: 2.158565  [57664/60000]
Test Error: 
 Accuracy: 50.3%, Avg loss: 2.145616 

Epoch 2
-------------------------------
loss: 2.151691  [   64/60000]
loss: 2.140949  [ 6464/60000]
loss: 2.079285  [12864/60000]
loss: 2.102268  [19264/60000]
loss: 2.053250  [25664/60000]
loss: 1.986810  [32064/60000]
loss: 2.017375  [38464/60000]
loss: 1.919152  [44864/60000]
loss: 1.926799  [51264/60000]
loss: 1.879050  [57664/60000]
Test Error: 
 Accuracy: 58.5%, Avg loss: 1.856283 

Epoch 3
-------------------------------
loss: 1.881377  [   64/60000]
loss: 1.856607  [ 6464/60000]
loss: 1.726216  [12864/60000]
loss: 1.783997  [19264/60000]
loss: 1.686481  [25664/60000]
loss: 1.628761  [32064/600

epochs = 5 (学习周期):
一个epoch代表大厨把整个训练食材库的所有菜都做了一遍。
epochs = 5 就是让大厨把整个菜谱从头到尾学习5遍，每一遍都会比上一遍做得更好。
小结： 训练就是一个 “做菜 -> 接受差评 -> 反思 -> 调整手法 -> 再做菜” 的循环过程。


In [23]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


torch.save(model.state_dict(), "model.pth") (记录核心手法):
model.state_dict() 就是获取大厨所有核心的烹饪手法和参数（权重和偏置），它是一个字典。
torch.save(...) 就是把这个“手法字典”保存成一个文件 model.pth。这就像是把菜谱的核心技巧记录在一张秘方上。

In [24]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

model.load_state_dict(torch.load("model.pth")) (回忆起手法):
model = NeuralNetwork(): 我们先找来一个新的、什么都不会的学徒（一个新的模型实例）。
torch.load("model.pth"): 从秘方文件中读取之前保存的“手法字典”。
model.load_state_dict(...): “学徒，快把这张秘方上的所有手法都记到脑子里！” 于是，这个新模型瞬间就拥有了之前训练好的所有能力。

In [25]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


model.eval() 和 with torch.no_grad() (正式营业):
现在大厨已经学成，可以正式营业（进行预测）了。我们让他进入评估模式，并且告诉他“只管做菜，不用再反思了”（关闭梯度计算），这样效率最高。

**Learn the Basics** \|\| [Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Learn the Basics
================

Authors: [Suraj Subramanian](https://github.com/subramen), [Seth
Juarez](https://github.com/sethjuarez/), [Cassie
Breviu](https://github.com/cassiebreviu/), [Dmitry
Soshnikov](https://soshnikov.com/), [Ari
Bornstein](https://github.com/aribornstein/)

Most machine learning workflows involve working with data, creating
models, optimizing model parameters, and saving the trained models. This
tutorial introduces you to a complete ML workflow implemented in
PyTorch, with links to learn more about each of these concepts.

We\'ll use the FashionMNIST dataset to train a neural network that
predicts if an input image belongs to one of the following classes:
T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker,
Bag, or Ankle boot.

[This tutorial assumes a basic familiarity with Python and Deep Learning
concepts.]{.title-ref}

Running the Tutorial Code
-------------------------

You can run this tutorial in a couple of ways:

-   **In the cloud**: This is the easiest way to get started! Each
    section has a \"Run in Microsoft Learn\" and \"Run in Google Colab\"
    link at the top, which opens an integrated notebook in Microsoft
    Learn or Google Colab, respectively, with the code in a fully-hosted
    environment.
-   **Locally**: This option requires you to setup PyTorch and
    TorchVision first on your local machine ([installation
    instructions](https://pytorch.org/get-started/locally/)). Download
    the notebook or copy the code into your favorite IDE.

How to Use this Guide
---------------------

If you\'re familiar with other deep learning frameworks, check out the
[0. Quickstart](quickstart_tutorial.html) first to quickly familiarize
yourself with PyTorch\'s API.

If you\'re new to deep learning frameworks, head right into the first
section of our step-by-step guide: [1. Tensors](tensorqs_tutorial.html).

::: {.toctree maxdepth="2" hidden=""}
quickstart\_tutorial tensorqs\_tutorial data\_tutorial
transforms\_tutorial buildmodel\_tutorial autogradqs\_tutorial
optimization\_tutorial saveloadrun\_tutorial
:::
